In [3]:
import os
from scripts.globals import *
import pandas as pd

In [15]:
df = pd.read_csv("data/sample_vcfs/sample.vcf", sep="\t", header=None,  names=["chr", "pos", "id", "ref", "alt"])

df.head()

,chr,pos,id,ref,alt
0,1,24255616,PD4120aC2,A,T


In [6]:
coords = pd.read_csv("data/mirna_coordinates/grch37_coordinates.csv")

coords.head()

,mirna_accession,mirna_name,chr,start,end,strand
0,MIMAT0027618,hsa-miR-6859-5p,1,17409,17431,-
1,MIMAT0027619,hsa-miR-6859-3p,1,17369,17391,-
2,MIMAT0005890,hsa-miR-1302,1,30438,30458,+
3,MIMAT0027618,hsa-miR-6859-5p,2,114353582,114353604,+
4,MIMAT0027619,hsa-miR-6859-3p,2,114353622,114353634,+


In [16]:
from scripts.pipeline import *

invalid = "invalid.txt"
df = validate_ref_nucleotides(df, invalid)  


In [17]:
df['is_mirna'] = 0
df['mirna_accession'] = None
df["pos"] = df["pos"].astype(int)

In [18]:
df

,chr,pos,id,ref,alt,ref_len,alt_len,is_mirna,mirna_accession
0,1,24255616,PD4120aC2,A,T,1,1,0,None


In [19]:
coords

,mirna_accession,mirna_name,chr,start,end,strand
0,MIMAT0027618,hsa-miR-6859-5p,1,17409,17431,-
1,MIMAT0027619,hsa-miR-6859-3p,1,17369,17391,-
2,MIMAT0005890,hsa-miR-1302,1,30438,30458,+
3,MIMAT0027618,hsa-miR-6859-5p,2,114353582,114353604,+
4,MIMAT0027619,hsa-miR-6859-3p,2,114353622,114353634,+
...,...,...,...,...,...,...
2876,MIMAT0005829,hsa-miR-1184,X,154687238,154687260,+
2877,MIMAT0018119,hsa-miR-3690,Y,1362820,1362842,+
2878,MIMAT0023714,hsa-miR-6089,Y,2477270,2477293,+
2879,MIMAT0039763,hsa-miR-9985,Y,4474181,4474199,+


In [22]:
mutation_start

24255616

In [47]:
coords.loc[(coords.start <= mutation_start) & (coords.end >= mutation_start) & coords.chr.astype(str) == mutation_chr.astype(str)]

AttributeError: 'int' object has no attribute 'astype'

In [43]:
coords.loc[(coords.start <= mutation_start) & (coords.end >= mutation_start) & (coords.chr == mutation_chr)]


,mirna_accession,mirna_name,chr,start,end,strand


In [31]:
matching_rnas = coords.loc[(coords['chr'] == mutation_chr) &
                            (coords['start'] <= mutation_start) &
                            (coords['end'] >= mutation_start)]

matching_rnas

,mirna_accession,mirna_name,chr,start,end,strand


In [34]:
for index, row in df.iterrows():
    mutation_chr = row['chr'].astype(str)
    print(mutation_chr)
    mutation_start = row['pos']
    print(mutation_start)
    
    
    
    # Find matching miRNAs
    matching_rnas = coords.loc[(coords['chr'] == mutation_chr) & (coords['start'] <= mutation_start) & (coords['end'] >= mutation_start)]

    if not matching_rnas.empty:
        # Update the 'is_mirna' and 'mirna_accession' columns
        df.at[index, 'is_mirna'] = 1
        df.at[index, 'mirna_accession'] = matching_rnas['mirna_accession'].values[0]

1
24255616


In [35]:
df

,chr,pos,id,ref,alt,ref_len,alt_len,is_mirna,mirna_accession
0,1,24255616,PD4120aC2,A,T,1,1,0,None


In [49]:
def generate_is_mirna_column(df, grch):
    """
    Adds two columns to the input DataFrame:
    'is_mirna': 1 if the mutation falls within a miRNA region, 0 otherwise
    'mirna_accession': the accession number of the miRNA if 'is_mirna' is 1, None otherwise

    Args:
        df (pandas.DataFrame): The input DataFrame containing mutation data
        grch (int): The genome reference coordinate system version (e.g., 37, 38)

    Returns:
        pandas.DataFrame: The input DataFrame with two additional columns ('is_mirna' and 'mirna_accession')
    """
    # Construct the miRNA coordinates file path
    mirna_coords_file = os.path.join(
        MIRNA_COORDS_DIR, f"grch{grch}_coordinates.csv")

    # Load miRNA coordinates
    coords = pd.read_csv(mirna_coords_file)

    # Initialize new columns
    df['is_mirna'] = 0
    df['mirna_accession'] = None

    # Convert datatypes for both dataframes
    df['chr'] = df['chr'].astype(str)
    df['pos'] = df['pos'].astype(int)
    
    coords['chr'] = coords['chr'].astype(str)
    coords['start'] = coords['start'].astype(int)
    coords['end'] = coords['end'].astype(int)

    # Iterate over each mutation in the mutations dataframe
    for index, row in df.iterrows():
        mutation_chr = row['chr']
        mutation_start = row['pos']

        # Find matching miRNAs
        matching_rnas = coords.loc[(coords['chr'] == mutation_chr) &
                                 (coords['start'] <= mutation_start) &
                                 (coords['end'] >= mutation_start)]

        if not matching_rnas.empty:
            # Update the 'is_mirna' and 'mirna_accession' columns
            df.at[index, 'is_mirna'] = 1
            df.at[index, 'mirna_accession'] = matching_rnas['mirna_accession'].values[0]

    return df


In [51]:
df = generate_is_mirna_column(df, grch=37)


In [52]:
df

,chr,pos,id,ref,alt,ref_len,alt_len,is_mirna,mirna_accession
0,1,24255616,PD4120aC2,A,T,1,1,1,MIMAT0018932
